Exercicio 10


In [1]:
import sys
print(sys.executable)
print(sys.version)


c:\Users\elidi\anaconda3\python.exe
3.13.9 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 19:09:58) [MSC v.1929 64 bit (AMD64)]


In [4]:
import sys
from pathlib import Path

# add /src to python path
project_root = Path().resolve().parent
src_path = project_root / "src"
sys.path.append(str(src_path))

print("Added to path:", src_path)


Added to path: C:\Users\elidi\si\src


In [17]:
import pandas as pd
import numpy as np

from si.io.csv_file import read_csv
from si.data.dataset import Dataset
from si.model_selection.split import train_test_split

from si.models.knn_classifier import KNNClassifier
from si.models.logistic_regression import LogisticRegression
from si.models.decision_tree_classifier import DecisionTreeClassifier
from si.ensemble.stacking_classifier import StackingClassifier

In [18]:
# Load dataset
dataset = read_csv('../datasets/breast_bin/breast-bin.csv', features=False, label=True)


In [19]:
# split dataset
train_dataset, test_dataset = train_test_split(dataset)
test_dataset.y

array([0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 1.])

In [20]:
knn_1 = KNNClassifier(k=3)
log_reg = LogisticRegression()
tree = DecisionTreeClassifier(max_depth=5)


In [21]:
final_knn = KNNClassifier(k=5)


In [22]:
stacking = StackingClassifier(
    base_models=[knn_1, log_reg, tree],
    meta_model=final_knn
)


In [23]:
stacking.fit(train_dataset)


In [24]:
score = stacking.score(test_dataset)
preds = stacking.predict(test_dataset)

print("\n--- StackingClassifier Results ---")
print("Accuracy on test set:", score)
print("First 10 predictions:", preds[:10])
print("First 10 true labels:", test_dataset.y[:10])



--- StackingClassifier Results ---
Accuracy on test set: 0.9784172661870504
First 10 predictions: [1. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
First 10 true labels: [0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]


Exercicio 11

In [25]:
import numpy as np

from si.io.csv_file import read_csv
from si.models.logistic_regression import LogisticRegression
from si.metrics.accuracy import accuracy
from si.model_selection.randomized_search import randomized_search_cv


In [27]:
dataset1 = read_csv('../datasets/breast_bin/breast-bin.csv', features=False, label=True)

print("Dataset shape:", dataset1.shape())
print("Classes:", np.unique(dataset1.y))



Dataset shape: (698, 9)
Classes: [0. 1.]


In [28]:
model = LogisticRegression()


In [29]:
param_grid = {
    "l2_penalty": np.linspace(1, 10, 10),
    "alpha": np.linspace(0.001, 0.0001, 100),
    "max_iter": np.linspace(1000, 2000, 200).astype(int)
}


In [30]:
results = randomized_search_cv(
    model=model,
    dataset=dataset1,
    hyperparameter_grid=param_grid,
    scoring=accuracy,
    cv=3,
    n_iter=10
)


In [31]:
print("\n--- RANDOMIZED SEARCH RESULTS ---")
print("Best Score:", results["best_score"])
print("Best Hyperparameters:")
for param, value in results["best_hyperparameters"].items():
    print(f"  {param}: {value}")



--- RANDOMIZED SEARCH RESULTS ---
Best Score: 0.9683908045977011
Best Hyperparameters:
  l2_penalty: 10.0
  alpha: 0.001
  max_iter: 1633
